In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info_from_webpage as extra_info
import parameters as params

In [ ]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [ ]:
# extract text from given .pdf file
def pdf2text(pdf_path, text_path, page_start, page_end):
    try:
        # creating a pdf reader object
        reader = PyPDF2.PdfReader(pdf_path)
        
        # printing number of pages in pdf file
        page_max = len(reader.pages)
        
        # getting a specific page from the pdf file
        text = ""
        
        for i in range(page_start, page_end + 1):
            # print(i)
            page = reader.pages[i]
            text = text + "".join(page.extract_text().splitlines())

        with open(text_path, "w") as f:
            f.write(text)
        f.close()

        return True
    except:
        return False
# --------------------start of test code--------------------
# ind = 1
# pdf_file_name = str(index) + ".pdf"
# pdf_folder_path = fpath.litera_pdf_folder
# pdf_path = os.path.join(pdf_folder_path, pdf_file_name)

# text_folder_path = fpath.litera_text_folder
# text_file_name = pdf_file_name.split(".pdf")[0] + ".txt"
# text_path = os.path.join(text_folder_path, text_file_name)

# start_page = 0
# end_page = 1
# if not pdf2text(pdf_path, text_path, start_page, end_page):
#     print("Error: pdf2text() failed")
# ---------------------end of test code---------------------

In [ ]:
def rename_pdf(newname, folder_of_download, time_to_wait=60):
    time_counter = 0
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa :   os.path.getctime(os.path.join(folder_of_download,xa)))
    while '.part' in filename:
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
    filename = max([f for f in os.listdir(folder_of_download)], key=lambda xa :   os.path.getctime(os.path.join(folder_of_download,xa)))
    os.rename(os.path.join(folder_of_download, filename), os.path.join(folder_of_download, newname))

In [ ]:
# download pdf to specified folder given pdf_url and ind
def download_pdf(pdf_url, pdf_folder_path):  
    url = pdf_url
    if url != url:
        raise Exception("pdf_url is np.nan")
    else:
        source = url.split("://")[1].split("/")[0]
    
    # pdf_path = os.path.join(pdf_folder_path, file_name)
      
    # get the pdf content
    # options = webdriver.ChromeOptions()
    # options.add_experimental_option('prefs', {
    # "download.default_directory": pdf_folder_path, #Change default directory for downloads
    # "download.prompt_for_download": False, #To auto download the file
    # "download.directory_upgrade": True,
    # "download.open_pdf_in_system_reader": False,
    # "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
    # })
    # driver = webdriver.Chrome(options=options)
    # # response = driver.get(pdf_url)
    # driver.navigate().to(url)
    # time.sleep(5)
    if source == "www.sciencedirect.com":
        os.environ['WDM_LOG'] = '0'
        options = Options()
        options.add_argument('--headless')
        driver = webdriver.Chrome()
        driver.get(url)
        element = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='ViewPDF']/a")))
        element.click()
        time.sleep(10)
    else: 
        options = webdriver.ChromeOptions()
        options.add_experimental_option('prefs', {
        "download.default_directory": pdf_folder_path, #Change default directory for downloads
        "download.prompt_for_download": False, #To auto download the file
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True #It will not show PDF directly in chrome
        })
        driver = webdriver.Chrome(options=options)
        response = driver.get(url)
        time.sleep(10)
    driver.close()

    # try:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         driver.close()
    #         return True
    # except:
    #     print(f'Failed downloading PDF:', url)
    #     driver.close()
    #     # print(f'HTTP response status code: {response.status_code}')
    #     return False

    # response = requests.get(pdf_url, headers=plib.headers)
    # response = requests.get(url, stream=True, allow_redirects=True, headers=plib.headers)
    
    # # download the .pdf file to the pdf_file_path folder
    # # write content in pdf file
    # pdf_path = os.path.join(pdf_folder_path, file_name)
    
    # if response.status_code == 200:
    #     with open(pdf_path, 'wb') as pdf_object:
    #         pdf_object.write(response.content)
    #         return True
    # else:
    #     print(f'Failed downloading PDF:' + 'pdf_url')
    #     print(f'HTTP response status code: {response.status_code}')
        # return False
# --------------------start of test code--------------------
# # pdf_url = 'https://www.sciencedirect.com/science/article/pii/S0896627320300052/pdfft?md5=3f0648c6385e6fae3a5a73b053903014&pid=1-s2.0-S0896627320300052-main.pdf'
# # pdf_url = "https://www.sciencedirect.com/science/article/pii/S1053811909013299?via%3Dihub"
# # pdf_url = "https://linkinghub.elsevier.com/retrieve/pii/S1053811909013299"
# # pdf_url = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1181753/pdf/jphysiol00456-0559.pdf"
# pdf_url = "https://www.sciencedirect.com/science/article/pii/S0896627320300052?via%3Dihub"

# # wiley.com
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.903130106"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902770204"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdf/10.1002/cne.902970309"
# # pdf_url = "https://onlinelibrary.wiley.com/doi/pdfdirect/10.1002/cne.902970309"

# # physiology.org
# # pdf_url = "https://journals.physiology.org/doi/pdf/10.1152/jn.2001.85.1.219"

# # springer.com
# # pdf_url = "https://link.springer.com/content/pdf/10.1007/BF00236173.pdf"

# # cell.com
# # pdf_url = "https://www.cell.com/neuron/pdf/S0896-6273(09)00170-6.pdf"

# # jneurosci.org
# # pdf_url = "https://www.jneurosci.org/content/jneuro/22/18/8117.full.pdf"
# # pdf_url, code = plib.get_final_redirected_url(pdf_url)
# # print(pdf_url)
# ind = 10
# file_name = str(ind) + ".pdf"
# pdf_folder_path = fpath.litera_pdf_folder
# download_pdf(pdf_url, pdf_folder_path)
# # rename_pdf(file_name, pdf_folder_path, time_to_wait=60)
# ---------------------end of test code---------------------

In [ ]:
# import pandas as pd
# import os
# import requests

# def get_pdf(doi, file_to_save_to):
#     url = 'http://api.elsevier.com/content/article/doi:'+doi+'?view=FULL'
#     headers = {
#         'X-ELS-APIKEY': "63f58b8b10cbc1bc923011c01c6301bb",
#         'Accept': 'application/pdf'
#     }
#     r = requests.get(url, stream=True, headers=headers)
#     if r.status_code == 200:
#         for chunk in r.iter_content(chunk_size=1024*1024):
#             file_to_save_to.write(chunk)
#         return True

# # doi_list = pd.read_excel('list.xls')
# # doi_list.columns = ['DOIs']
# count = 0
# # for doi in doi_list['DOIs']:
# doi = "10.1016/j.neuron.2020.01.005"
# # pdf = "10"
# ind = 10
# file_name = str(ind) + ".pdf"
# # if not os.path.exists(f'path/{pdf}.pdf'):
# with open(file_name, 'wb') as pdf_object:
#     get_pdf(doi, pdf_object)
# pdf_object.close

In [ ]:

def func_elsevier_com(url):
    # initialize
    info = {
        "doi": np.nan,
        "pmid": np.nan,
        "pmcid": np.nan,
        "title": np.nan,
        "abstract": np.nan,
        "keywords": np.nan,
        "pdf_link": np.nan
    }

    # set up the webdriver
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome()

    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver.get(url)
            time.sleep(10)
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # doi
    try:
        doi = driver.find_element(By.XPATH, "//a[@class='anchor doi anchor-default']/span").text.split("doi.org/")[1]
        doi = doi.strip()
    except:
        doi = np.nan
    if doi == doi:
        doi = doi.lower()

    # pmid, pmcid
    pmid = np.nan
    pmcid = np.nan

    # title
    try:
        title = driver.find_element(By.TAG_NAME, 'h1').find_element(By.XPATH, "//span[@class='title-text']").text
        title = title.strip()
    except:
        title = np.nan

    # abstract
    try:
        abstract = ""
        elems = driver.find_element(By.XPATH, "//div[@class='abstract author']").find_elements(By.TAG_NAME, "p")
        for elem in elems:
            abstract = abstract + elem.text + " "
        abstract = abstract.strip()
    except:
        abstract = np.nan
    
    # keywords
    try:
        keywords = ""
        elements = driver.find_element(By.XPATH, "//div[@class='keywords-section']").find_elements(By.XPATH, "//div[@class='keyword']")
        for element in elements:
            keywords = keywords + element.find_element(By.TAG_NAME, "span").text + ", "
        keywords = keywords.strip()
    except:
        keywords = np.nan

    # pdf_link
    try:
        pdf_link = driver.find_element(By.XPATH, "//li[@class='ViewPDF']/a").get_attribute('href')
        pdf_link = pdf_link.strip()
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
url = "https://linkinghub.elsevier.com/retrieve/pii/0006899395013385"
# url = "https://linkinghub.elsevier.com/retrieve/pii/S0891061898000222"
# url = "https://www.sciencedirect.com/science/article/pii/S0006322310010036?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S0165027017303631?via%3Dihub#abs0010"
# url = "https://linkinghub.elsevier.com/retrieve/pii/S0165027017303631"
# url = "https://www.sciencedirect.com/science/article/pii/S0079612308626783?via%3Dihub"
# url = "https://linkinghub.elsevier.com/retrieve/pii/0006899376902067"
# url = "https://www.sciencedirect.com/science/article/pii/0006899378911034?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0006899377904747?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0165017379900080?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/000689937990132X?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S2211124723008550?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S0960982215014190?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0006899375905296?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S0166432805800166?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/000689938690925X?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/abs/pii/S1042368018302602?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0006899367900042?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0165017380900028?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/B9780124077942000092?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0006899368900450?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/S0006322310010036?via%3Dihub"
# url = "https://www.sciencedirect.com/science/article/pii/0006899377907806?via%3Dihub"
info = func_elsevier_com(url)
print(info["doi"])
print(info["pmid"])
print(info["pmcid"])
print(info["title"])
print(info["abstract"])
print(info["keywords"])
print(info["pdf_link"])
# ---------------------end of test code---------------------